# Import libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0,'..')
#!pip install openpyxl

In [2]:
import os
import sys
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from functools import partial
from multiprocessing import Pool

from lhcsmapi.Time import Time
from lhcsmapi.metadata.SignalMetadata import SignalMetadata
from lhcsmapi.pyedsl.dbsignal.post_mortem.PmDbRequest import PmDbRequest
from lhcsmapi.analysis.RbCircuitQuery import RbCircuitQuery

from src.acquisitions.current_voltage_diode_leads_nxcals import CURRENT_VOLTAGE_DIODE_LEADS_NXCALS
from src.acquisitions.current_voltage_diode_leads_pm import CURRENT_VOLTAGE_DIODE_LEADS_PM
from src.acquisitions.ee_t_res_pm import EE_T_RES_PM
from src.acquisitions.ee_u_dump_res_pm import EE_U_DUMP_RES_PM
from src.acquisitions.leads import LEADS
from src.acquisitions.pc_pm import PC_PM
from src.acquisitions.qh_pm import QH_PM
from src.acquisitions.voltage_logic_iqps import VOLTAGE_LOGIC_IQPS
from src.acquisitions.voltage_logic_nqps import VOLTAGE_LOGIC_NQPS
from src.acquisitions.voltage_nxcals import VOLTAGE_NXCALS

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: htt

# Read the (clean) MP3 file

In [3]:
mp3_fpa_df_raw = pd.read_excel("./RB_TC_extract_2021_11_22.xlsx")
len(mp3_fpa_df_raw)

4409

In [4]:
# First row contains units, 9 rows contain only "Before Notebooks" and "After Notebooks" information, which need to be droped
mp3_fpa_df = mp3_fpa_df_raw.dropna(subset = ['Date (FGC)', 'Circuit Name'])
mp3_fpa_df_raw.iloc[~mp3_fpa_df_raw.index.isin(mp3_fpa_df.index)] #show droped rows

,Circuit Name,Circuit Family,Period,Date (FGC),Time (FGC),FPA Reason,Timestamp_PIC,Delta_t(FGC-PIC),Delta_t(EE_even-PIC),Delta_t(EE_odd-PIC),...,Outer cable type,I_Q_SM18,dI_Q_Acc,dI_Q_LHC,Comment,Analysis performed by,lhcsmapi version,lhcsm notebook version,Unnamed: 46,Unnamed: 47
0,NaN,RB,NaN,2008-04-01 00:00:00,13:14:15,NaN,NaN,[ms],[ms],[ms],...,NaN,[A],[A],[A],NaN,NaN,NaN,NaN,NaN,NaN
1,RB.A12,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
169,RB.A23,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
264,RB.A34,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
545,RB.A45,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
897,RB.A56,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1146,RB.A67,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1268,RB.A78,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1430,RB.A81,RB,Before Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Before Notebooks,NaN,NaN,NaN
1581,NaN,RB,After Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,After Notebooks,NaN,NaN,NaN


# Find FGC timestamp for each FPA event in MP3 excel

In [5]:
def find_real_fgc_timestamp(circuit_name, fgc_datetime):
    
    fgc_timestamp = Time.to_unix_timestamp(fgc_datetime)
    metadata_fgc = SignalMetadata.get_circuit_signal_database_metadata(
            'RB', circuit_name, 'PC', 'PM', timestamp_query=fgc_timestamp)

    one_sec_in_ns = 1e9 
    start_time = fgc_timestamp - one_sec_in_ns
    end_time = fgc_timestamp + one_sec_in_ns
 
    source_timestamp_fgc = PmDbRequest.find_events(metadata_fgc['source'],
                                                   metadata_fgc['system'],
                                                   metadata_fgc['className'],
                                                   t_start=start_time,
                                                   t_end=end_time)

    return [(circuit_name, el[1]) for el in source_timestamp_fgc]

# getting the list of timestamps
def get_fgc_timestamp(d):
    date_time_str = f"{d['Date (FGC)']} {d['Time (FGC)']}".replace("00:00:00 ","")
    real_fgc_timestamps = find_real_fgc_timestamp(d['Circuit Name'], date_time_str)
    if len(real_fgc_timestamps) > 1:
        print(f"Found more than one event for timestamp {date_time_str}")
    elif not real_fgc_timestamps:
        print(f"Found no event for timestamp {date_time_str}")
        return None
    _, real_fgc_timestamp = real_fgc_timestamps[0]
    return int(real_fgc_timestamp)

In [6]:
mp3_fpa_df['timestamp_fgc'] = mp3_fpa_df.apply(get_fgc_timestamp, axis=1)

Found no event for timestamp 2008-08-19 12:13:28
Found no event for timestamp 2008-08-19 18:53:12
Found no event for timestamp 2008-08-19 20:15:07
Found no event for timestamp 2008-08-19 20:15:07
Found no event for timestamp 2008-08-20 18:41:36
Found no event for timestamp 2008-08-22 09:59:25
Found no event for timestamp 2008-08-22 09:59:25
Found no event for timestamp 2008-08-22 09:59:25
Found no event for timestamp 2008-07-08 08:30:10
Found no event for timestamp 2008-07-08 14:39:16
Found no event for timestamp 2008-07-09 17:14:45
Found no event for timestamp 2008-07-11 14:42:00
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-07-15 21:20:30
Found no event for timestamp 2008-08-09 02:19:48
Found no event for timestamp 2008-09-07 17:34:03
Found no event for timestamp 2009-12-04 11:19:48
Found no event for t

Found no event for timestamp 2008-04-28 17:08:50
Found no event for timestamp 2008-04-28 17:08:50
Found no event for timestamp 2008-04-29 16:42:45
Found no event for timestamp 2008-04-29 16:42:45
Found no event for timestamp 2008-04-29 16:42:45
Found no event for timestamp 2008-04-29 16:42:45
Found no event for timestamp 2008-04-30 09:02:32
Found no event for timestamp 2008-04-30 09:02:32
Found no event for timestamp 2008-04-30 09:02:32
Found no event for timestamp 2008-04-30 09:02:32
Found no event for timestamp 2008-04-30 09:02:32
Found no event for timestamp 2008-05-06 18:12:16
Found no event for timestamp 2008-05-06 18:12:16
Found no event for timestamp 2008-05-06 18:12:16
Found no event for timestamp 2008-05-06 18:12:16
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for timestamp 2008-05-07 17:53:50
Found no event for t

Found no event for timestamp 2015-12-03 09:52:07
Found no event for timestamp 2015-12-03 09:52:07
Found no event for timestamp 2015-12-03 09:52:07
Found no event for timestamp 2008-06-30 11:40:04
Found no event for timestamp 2008-06-30 15:39:44
Found no event for timestamp 2008-07-01 09:58:44
Found no event for timestamp 2008-07-01 09:58:44
Found no event for timestamp 2008-07-01 09:58:44
Found no event for timestamp 2008-07-01 09:58:44
Found no event for timestamp 2008-07-01 09:58:44
Found no event for timestamp 2008-07-01 09:58:44
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:43:20.100000
Found no event for timestamp 2021-05-22 00:58:40.300000
Found no event for timestamp 2021-05-22 00:58:40.300000
Found no even


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Filter mp3 file

In [7]:
# dropping duplicates of timestamps
print(f"{len(mp3_fpa_df['timestamp_fgc'])} mp3 excel file entries")
mp3_fpa_df_unique = mp3_fpa_df.drop_duplicates(subset=['timestamp_fgc', 'Circuit Name']) ### you need timestamp + circuit !!! 

print(f"{len(mp3_fpa_df_unique)} unique fgc events") # 841 Primary quenches?
print(f"{len(mp3_fpa_df[mp3_fpa_df['Nr in Q event']==1])} mp3 primary quench entries") # 834 primary quenches with correct notes

4399 mp3 excel file entries
841 unique fgc events
918 mp3 primary quench entries


In [8]:
operation_periods =[
    'HWC 2007', 
    'HWC 2008', 
    'HWC 2009',
    'HWC 2010', 
    'HWC Oct 2014', 
    'HWC Dec 2014', 
    'HWC 2015', 
    'Operation 2015', 
    'HWC 2016',
    'Operation 2016',
    'HWC Dec 2016', 
    'HWC 2017', 
    'Operation 2017', 
    'Operation 2018',
    'HWC Dec 2018',
    'HWC 2021', 
]

In [9]:
fig, ax = plt.subplots(figsize=(10,10))
width=0.3
mp3_fpa_df.Period.value_counts()[operation_periods].plot(ax=ax, kind='barh', width=width, position=0, label='mp3 fpa entries')
mp3_fpa_df_unique.Period.value_counts()[operation_periods].plot(ax=ax, kind='barh', width=width, position=1, color='red', label='unique fgc event')
#mp3_fpa_df[mp3_fpa_df['Nr in Q event']==1].Period.value_counts()[operation_periods].plot(ax=ax, kind='barh', width=width, position=2, color='orange', label='unique fgc event')
plt.legend()

In [10]:
# mp3 primary quench entries with no unique fgc entry
mp3_fpa_df_missing = mp3_fpa_df[mp3_fpa_df['Nr in Q event']==1][~mp3_fpa_df[mp3_fpa_df['Nr in Q event']==1].index.isin(mp3_fpa_df_unique.index)] 
mp3_fpa_df_missing.Period.value_counts()

HWC 2008          69
HWC 2007           4
HWC 2021           4
HWC 2010           2
HWC 2015           2
HWC 2009           1
Operation 2017     1
HWC Dec 2018       1
Name: Period, dtype: int64

In [11]:
mp3_fpa_df_missing[mp3_fpa_df_missing['Period'] == 'HWC 2021']

,Circuit Name,Circuit Family,Period,Date (FGC),Time (FGC),FPA Reason,Timestamp_PIC,Delta_t(FGC-PIC),Delta_t(EE_even-PIC),Delta_t(EE_odd-PIC),...,I_Q_SM18,dI_Q_Acc,dI_Q_LHC,Comment,Analysis performed by,lhcsmapi version,lhcsm notebook version,Unnamed: 46,Unnamed: 47,timestamp_fgc
3050,RB.A12,RB,HWC 2021,2021-05-22 00:00:00,00:43:20.100000,Magnet quench,2021-05-22 00:43:20,31,592,96,...,11313,550,NaN,NaN,emm,1.5.13,1.5.63,NaN,NaN,NaN
3056,RB.A23,RB,HWC 2021,2021-05-22 00:00:00,00:58:40.300000,Magnet quench,2021-05-22 00:58:40,27,594,91,...,11213,261,NaN,NaN,emm,1.5.13,1.5.63,NaN,NaN,NaN
3529,RB.A56,RB,HWC 2021,2021-06-15 00:00:00,00:10:53.600000,Magnet quench,00:10:53.500000,20,596,96,...,12394,-1050,NaN,NaN,emm,1.5.14,1.5.64,NaN,NaN,NaN
3569,RB.A56,RB,HWC 2021,2021-06-19 00:00:00,00:25:03.400000,Magnet quench,00:25:03.400000,31,596,95,...,12307,-890,NaN,NaN,hagen,1.5.14,1.5.64,NaN,NaN,NaN


# Select events to download

In [13]:
# Time.to_string(1514761200000000000), Time.to_unix_timestamp('2018-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= 1388530800000000000].reset_index(drop=True) ### we want to analyse >= 2017.01.01
len(mp3_fpa_df_period)

824

In [14]:
def log_failed_files(circuit_type, circuit_name, timestamp_fgc):
    identifier = {'circuit_type': circuit_type,
                  'circuit_name': circuit_name,
                  'timestamp_fgc': timestamp_fgc}
    filename=Path('failed_files.csv')
    if not filename.is_file():
        df = pd.DataFrame(identifier, index=[0])
    else:
        df = pd.read_csv(filename)
        df_new = pd.DataFrame(identifier, index=[0])
        df = pd.concat([df, df_new], axis=0)
    df.to_csv(filename, index=False)

In [15]:
signal_groups = [PC_PM, EE_U_DUMP_RES_PM, EE_T_RES_PM, VOLTAGE_NXCALS, VOLTAGE_LOGIC_IQPS, VOLTAGE_LOGIC_NQPS, QH_PM, 
                 CURRENT_VOLTAGE_DIODE_LEADS_PM, CURRENT_VOLTAGE_DIODE_LEADS_NXCALS, LEADS]

hdf_dir = Path('data')
context_path = Path('context_data.csv')

In [ ]:
for index, row in mp3_fpa_df_period.iterrows():
    circuit_type = row['Circuit Family']
    circuit_name = row['Circuit Name']
    timestamp_fgc = int(row['timestamp_fgc'])
    try:
        for signal_group in signal_groups:
            group = signal_group(circuit_type, circuit_name, timestamp_fgc, spark=spark)
            group.to_hdf5(hdf_dir=hdf_dir, context_path=context_path)
            print(f"successfully downloaded: {str(signal_group)}")
    except:
        log_failed_files(circuit_type, circuit_name, timestamp_fgc)
        print(f"failed to download: {str(signal_group)}")

	Querying PM event signal(s) IAB.I_A, STATUS.I_EARTH_PCNT, STATUS.I_MEAS, STATUS.I_REF, IEARTH.IEARTH for system: RPTE.UA23.RB.A12, className: 51_self_pmd, source: FGC at 2014-12-11 21:45:14.580
successfully downloaded: <class 'src.acquisitions.pc_pm.PC_PM'>
	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: RR17.RB.A12, UA23.RB.A12 from 2014-12-11 21:45:04.580 to 2014-12-11 22:01:54.580
	Querying PM event signal(s) RR17.RB.A12:U_DUMP_RES, UA23.RB.A12:U_DUMP_RES for system: RR17.RB.A12, UA23.RB.A12, className: DQAMSNRB, source: QPS at 2014-12-11 21:45:14.877


Post Mortem returned no data for the following query: http://pm-api-pro/v2/pmdata/signal?system=QPS&className=DQAMSNRB&source=UA23.RB.A12&timestampInNanos=1418330714877000000&signal=UA23.RB.A12:U_DUMP_RES


successfully downloaded: <class 'src.acquisitions.ee_u_dump_res_pm.EE_U_DUMP_RES_PM'>
	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: RR17.RB.A12, UA23.RB.A12 from 2014-12-11 21:45:04.580 to 2014-12-11 22:01:54.580
	Querying PM event signal(s) UA23.RB.A12:T_RES_BODY_2, RR17.RB.A12:T_RES_BODY_2, UA23.RB.A12:T_RES_BODY_1, RR17.RB.A12:T_RES_BODY_1, RR17.RB.A12:T_RES_BODY_3... for system: RR17.RB.A12, UA23.RB.A12..., className: DQAMSNRB..., source: QPS... at 2014-12-11 21:45:14.877


Post Mortem returned no data for the following query: http://pm-api-pro/v2/pmdata/signal?system=QPS&className=DQAMSNRB&source=UA23.RB.A12&timestampInNanos=1418330714877000000&signal=UA23.RB.A12:T_RES_BODY_1
Post Mortem returned no data for the following query: http://pm-api-pro/v2/pmdata/signal?system=QPS&className=DQAMSNRB&source=UA23.RB.A12&timestampInNanos=1418330714877000000&signal=UA23.RB.A12:T_RES_BODY_2
Post Mortem returned no data for the following query: http://pm-api-pro/v2/pmdata/signal?system=QPS&className=DQAMSNRB&source=UA23.RB.A12&timestampInNanos=1418330714877000000&signal=UA23.RB.A12:T_RES_BODY_3


successfully downloaded: <class 'src.acquisitions.ee_t_res_pm.EE_T_RES_PM'>
	Querying NXCALS signal(s) MB.A10R1:U_DIODE_RB, MB.A9R1:U_DIODE_RB, MB.B9R1:U_DIODE_RB, MB.A8R1:U_DIODE_RB, MB.B8R1:U_DIODE_RB... from 2014-12-11 21:44:24.580 to 2014-12-11 21:53:34.580
	Querying NXCALS signal(s) MB.C12R1:U_EARTH_RB, MB.B8R1:U_EARTH_RB, MB.B11R1:U_EARTH_RB, MB.A8R1:U_EARTH_RB, MB.A10R1:U_EARTH_RB... from 2014-12-11 21:44:24.580 to 2014-12-11 21:53:34.580
successfully downloaded: <class 'src.acquisitions.voltage_nxcals.VOLTAGE_NXCALS'>
	Querying PM event timestamps for system: QPS, className: DQAMCNMB_PMSTD, source: * from 2014-12-11 21:44:24.580 to 2014-12-11 21:53:34.580


...

successfully downloaded: <class 'src.acquisitions.leads.LEADS'>
	Querying PM event signal(s) IAB.I_A, STATUS.I_EARTH_PCNT, STATUS.I_MEAS, STATUS.I_REF, IEARTH.IEARTH for system: RPTE.UA43.RB.A34, className: 51_self_pmd, source: FGC at 2015-03-15 03:30:29.160
successfully downloaded: <class 'src.acquisitions.pc_pm.PC_PM'>
	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: UA43.RB.A34, UJ33.RB.A34 from 2015-03-15 03:30:19.160 to 2015-03-15 03:47:09.160
	Querying PM event signal(s) UA43.RB.A34:U_DUMP_RES, UJ33.RB.A34:U_DUMP_RES for system: UA43.RB.A34, UJ33.RB.A34, className: DQAMSNRB, source: QPS at 2015-03-15 03:30:29.482


Post Mortem returned no data for the following query: http://pm-api-pro/v2/pmdata/signal?system=QPS&className=DQAMSNRB&source=UA43.RB.A34&timestampInNanos=1426386629482000000&signal=UA43.RB.A34:U_DUMP_RES


successfully downloaded: <class 'src.acquisitions.ee_u_dump_res_pm.EE_U_DUMP_RES_PM'>
	Querying PM event timestamps for system: QPS, className: DQAMSNRB, source: UA43.RB.A34, UJ33.RB.A34 from 2015-03-15 03:30:19.160 to 2015-03-15 03:47:09.160
	Querying PM event signal(s) UJ33.RB.A34:T_RES_BODY_1, UA43.RB.A34:T_RES_BODY_2, UJ33.RB.A34:T_RES_BODY_3, UJ33.RB.A34:T_RES_BODY_2, UA43.RB.A34:T_RES_BODY_1... for system: UA43.RB.A34, UJ33.RB.A34..., className: DQAMSNRB..., source: QPS... at 2015-03-15 03:30:29.482


Post Mortem returned no data for the following query: http://pm-api-pro/v2/pmdata/signal?system=QPS&className=DQAMSNRB&source=UA43.RB.A34&timestampInNanos=1426386629482000000&signal=UA43.RB.A34:T_RES_BODY_1
Post Mortem returned no data for the following query: http://pm-api-pro/v2/pmdata/signal?system=QPS&className=DQAMSNRB&source=UA43.RB.A34&timestampInNanos=1426386629482000000&signal=UA43.RB.A34:T_RES_BODY_2
Post Mortem returned no data for the following query: http://pm-api-pro/v2/pmdata/signal?system=QPS&className=DQAMSNRB&source=UA43.RB.A34&timestampInNanos=1426386629482000000&signal=UA43.RB.A34:T_RES_BODY_3


successfully downloaded: <class 'src.acquisitions.ee_t_res_pm.EE_T_RES_PM'>
	Querying NXCALS signal(s) MB.B9R3:U_DIODE_RB, MB.A8R3:U_DIODE_RB, MB.A10R3:U_DIODE_RB, MB.B8R3:U_DIODE_RB, MB.A9R3:U_DIODE_RB... from 2015-03-15 03:29:39.160 to 2015-03-15 03:38:49.160
	Querying NXCALS signal(s) MB.A12R3:U_EARTH_RB, MB.B9R3:U_EARTH_RB, MB.A9R3:U_EARTH_RB, MB.A14R3:U_EARTH_RB, DFLAS.7R3.1:U_EARTH_RB... from 2015-03-15 03:29:39.160 to 2015-03-15 03:38:49.160


# Analyze downloaded files

In [14]:
context_df = pd.read_csv("./context_data.csv")
len(context_df)

18088

In [15]:
context_df

,circuit_type,circuit_name,timestamp_fgc,STATUS.I_MEAS,IAB.I_A,IEARTH.IEARTH,STATUS.I_EARTH_PCNT,STATUS.I_REF,RR17.RB.A12:U_DUMP_RES,RR17.RB.A12:T_RES_BODY_1,...,B12L4:ST_NQD0,B12L4:ST_MAGNET_OK,B11L4:U_QS0,B11L4:U_1,B11L4:U_2,B11L4:ST_NQD0,B11L4:ST_MAGNET_OK,A18L4:U_QS0,A18L4:U_1,A18L4:U_2
0,RB,RB.A12,1418330714580000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RB,RB.A12,1418384764500000000,21000.0,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RB,RB.A12,1418384764500000000,NaN,8192.0,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RB,RB.A12,1418384764500000000,NaN,NaN,1024.0,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RB,RB.A12,1418384764500000000,NaN,NaN,NaN,21000.0,21000.0,1198.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18083,RB,RB.A34,1426750907160000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18084,RB,RB.A34,1426750907160000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18085,RB,RB.A34,1426750907160000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18086,RB,RB.A34,1426750907160000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
